<a href="https://colab.research.google.com/github/jPruim/cs344/blob/master/Project/Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np                                       # fast vectors and matrices
import matplotlib.pyplot as plt                          # plotting

from IPython.display import Audio

from intervaltree import Interval,IntervalTree

%matplotlib inline
fs = 44100      # samples/second


In [0]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1fZkouwSFidMW-dr6HbJjkkMDx2wGR4XQ'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [0]:
train_data = np.load(open('../musicnet.npz','rb'))

In [0]:
print 'Number of recordings: ' + str(len(train_data.files))
print 'Example MusicNet ids: ' + str(train_data.keys()[0:5])

In [0]:
X,Y = train_data['2494'] # data X and labels Y for recording id 2494
print type(X)
print type(Y)

In [0]:
fig = plt.figure()
fig.set_figwidth(20)
fig.set_figheight(2)
plt.plot(X[0:30*fs],color=(41/255.,104/255.,168/255.))
fig.axes[0].set_xlim([0,30*fs])
fig.axes[0].set_xlabel('sample (44,100Hz)')
fig.axes[0].set_ylabel('amplitude')

In [0]:
Audio(X[0:30*fs],rate=fs)

In [0]:
print 'Notes present at sample ' + str(fs*5) + ' (5 seconds): ' + str(len(Y[fs*5]))


In [0]:
print 'Notes present at sample ' + str(fs*4) + ' (4 seconds): ' + str(len(Y[fs*4]))


In [0]:
(start,end,(instrument,note,measure,beat,note_value)) = sorted(Y[fs*5])[0]
print ' -- An example of a MusicNet label -- '
print ' Start Time:                          ' + str(start)
print ' End Time:                            ' + str(end)
print ' Instrument (MIDI instrument code):   ' + str(instrument)
print ' Note (MIDI note code):               ' + str(note)
print ' Measure:                             ' + str(measure)
print ' Beat (0 <= beat < 1):                ' + str(beat)
print ' Note Value:                          ' + str(note_value)

In [0]:

stride = 512                         # 512 samples between windows
wps = fs/float(stride)               # ~86 windows/second
Yvec = np.zeros((int(30*wps),128))   # 128 distinct note labels
colors = {41 : .33, 42 : .66, 43 : 1}

for window in range(Yvec.shape[0]):
    labels = Y[window*stride]
    for label in labels:
        Yvec[window,label.data[1]] = colors[label.data[0]]

In [0]:
fig = plt.figure(figsize=(20,5))
plt.imshow(Yvec.T,aspect='auto',cmap='ocean_r')
plt.gca().invert_yaxis()
fig.axes[0].set_xlabel('window')
fig.axes[0].set_ylabel('note (MIDI code)')

In [0]:
mix_size=4096

labels = np.zeros(X.shape)
for (onset,offset,label) in sorted(Y):
    freq = 440.*2**((label[1] - 69.)/12.)
    mark = np.sin(freq*2.*np.pi*np.arange(0,mix_size)/float(fs))
    if onset + len(mark) < len(labels):
        labels[onset:onset+mix_size] += mark
        
labels /= np.max(labels)

mix = .8*labels + .2*X

In [0]:
Audio(mix[0:30*fs],rate=fs)
